In [1]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/env_0324/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


### LlamaParser

In [1]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

In [3]:
import os
from tqdm import tqdm
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 1. 파서 설정
parser = LlamaParse(
    result_type="markdown",
    num_workers=4,
    verbose=True,
    language="ko",
)

# 2. pdf2 폴더에 있는 PDF 리스트 만들기
pdf_dir = "pdf2"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

# 3. tqdm으로 진행상황 보면서 하나씩 처리
documents = []
for file in tqdm(pdf_files, desc="📄 파싱 중"):
    reader = SimpleDirectoryReader(input_files=[file], file_extractor={".pdf": parser})
    docs = reader.load_data()
    documents.extend(docs)

print(f"\n✅ 총 {len(documents)}개의 문서가 파싱되었습니다.")


📄 파싱 중:   0%|          | 0/4 [00:00<?, ?it/s]

Started parsing the file under job_id e9206efb-c52d-4054-9ce5-b05c51986e44
....

📄 파싱 중:  25%|██▌       | 1/4 [01:52<05:36, 112.20s/it]

Started parsing the file under job_id 7a9cf6e7-fc31-405d-9605-568d907d16a6
..........

📄 파싱 중:  50%|█████     | 2/4 [06:02<06:27, 193.62s/it]

Started parsing the file under job_id 62232b42-0d7a-41fc-afae-93ddd87db6a5


📄 파싱 중:  75%|███████▌  | 3/4 [06:22<01:54, 114.01s/it]

Started parsing the file under job_id c3757ad9-7586-484f-bd31-92162b2c2f09
..

📄 파싱 중: 100%|██████████| 4/4 [07:32<00:00, 113.18s/it]


✅ 총 795개의 문서가 파싱되었습니다.


In [4]:
len(documents)

795

In [5]:
documents[12]

Document(id_='744c8b48-dd6f-46d6-aead-3336dcd1405c', embedding=None, metadata={'file_path': 'pdf2/현대미술관연구 제 19집 (2008년).pdf', 'file_name': '현대미술관연구 제 19집 (2008년).pdf', 'file_type': 'application/pdf', 'file_size': 11079831, 'creation_date': '2025-03-24', 'last_modified_date': '2025-03-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='장품의 매각이 법적으로 가능하게 되므로 국가문화유산의 체계적 관리 문제가 제기될 수 있다. 법인화 시, 대두될 수 있는 이러한 여러 문제점들은 국가의 문화정책, 외교전략 면에서도 매우 심각하게 부각되므로 사전에 신중히 고려되어야 한다.\n\n결론적으로, 미술관·박물관은 변화하는 환경과 경제논리를 피할 수 없으나, 인프라가 조성되지 않은 현재의 상황에서는 국가가 안정적 재원을 제공하면서 주도적으로 문화를 육성하는 것이 필요하다. 법인화를 추진하더라도 행정개혁 논리가 아닌 문화개혁의 차원에서 충분한 준비기간을 두고 추진해야 한다. 일본의 경우만 해도

In [8]:
texts = [doc.text for doc in documents]
embeddings = embedding_model.encode(texts)

In [10]:
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore

embedding_dim = len(embeddings[0])
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings).astype(np.float32))  # 꼭 float32로!

# 5. Docstore & ID 매핑 (key는 str)
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
index_to_docstore_id = {i: str(i) for i in range(len(documents))}

# 6. FAISS 벡터스토어 생성
vectorstore = FAISS(
    embedding_function=embedding_model,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. 저장
vectorstore.save_local("faiss_artworks_llamaparse_pdf2")
print("🎉 FAISS DB 저장 완료 → faiss_artworks_llamaparse_pdf2")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


🎉 FAISS DB 저장 완료 → faiss_artworks_llamaparse_pdf2


### PDFParser

In [6]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1", device='cuda')


/home/chae/env_0324/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [128]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import pdfplumber
from langchain_core.documents import Document
from langchain.docstore.in_memory import InMemoryDocstore


# ✅ 2. PDF 문단을 하나씩 lazy하게 추출하는 함수
def extract_text_paragraphs_lazy(pdf_path):
    pdf_doc = fitz.open(pdf_path)
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf_doc)):
            page = pdf_doc[page_num]
            text = page.get_text("text")
            paragraphs = text.split("\n\n")

            tables = pdf.pages[page_num].extract_tables()
            table_texts = []
            if tables:
                for table in tables:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    table_texts.append(df.to_string())

            for para in paragraphs:
                full_text = para.strip() + "\n\n" + "\n\n".join(table_texts)
                if len(full_text.strip()) < 30:  # 너무 짧은 문단은 제외
                    continue
                yield Document(
                    page_content=full_text,
                    metadata={"page": page_num + 1, "source": os.path.basename(pdf_path)}
                )

In [129]:
def create_docstore(documents):
    """
    문서 객체를 InMemoryDocstore에 저장합니다.
    """
    docstore = InMemoryDocstore({i: doc for i, doc in enumerate(documents)})
    return docstore

In [201]:
import time,os
from langchain_community.vectorstores import FAISS

faiss_index = None
docstore_docs = {}
doc_id_counter = 0

# ✅ 4. PDF 디렉토리 순회
pdf_dir = "./pdf2"
for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        print(f"📄 Processing: {filename}")
        
        try:
            for doc in extract_text_paragraphs_lazy(pdf_path):
                text = doc.page_content
                embedding = embedding_model.encode([text])  # 문단 1개씩 임베딩

                if faiss_index is None:
                    dim = len(embedding[0])
                    faiss_index = faiss.IndexFlatL2(dim)

                faiss_index.add(np.array(embedding).astype(np.float32))
                docstore_docs[doc_id_counter] = doc
                doc_id_counter += 1

            print(f"✅ {filename} 처리 완료, 현재 총 문서 수: {doc_id_counter}")
            time.sleep(1)

        except Exception as e:
            print(f"❌ {filename} 처리 중 에러 발생: {e}")


📄 Processing: 현대미술관연구 제 19집 (2008년).pdf
✅ 현대미술관연구 제 19집 (2008년).pdf 처리 완료, 현재 총 문서 수: 178
📄 Processing: 현대미술관연구 제 17집(2006년).pdf


KeyboardInterrupt: 

In [202]:
from langchain_community.vectorstores import FAISS

docstore = InMemoryDocstore(docstore_docs)
index_to_docstore_id = {i: str(i) for i in docstore_docs.keys()}

vectorstore = FAISS(
    embedding_function=embedding_model,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [203]:
save_path = "faiss_artworks_0324_pdf2"
vectorstore.save_local(save_path)

print(f"🎉 저장 완료: {save_path}")

🎉 저장 완료: faiss_artworks_0324_pdf2


In [42]:
print("🧪 FAISS 객체 확인")
print("index:", type(vectorstore.index))
print("docstore:", type(vectorstore.docstore))
print("index_to_docstore_id:", type(vectorstore.index_to_docstore_id))

🧪 FAISS 객체 확인
index: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
docstore: <class 'langchain_community.docstore.in_memory.InMemoryDocstore'>
index_to_docstore_id: <class 'dict'>


In [204]:
docstore_docs

{0: Document(metadata={'page': 1, 'source': '현대미술관연구 제 19집 (2008년).pdf'}, page_content='현대미술관연구  1\n목  차\nⅠ. 미술관학 연구\n- 강승완 _ 미술관·박물관문화의 패러다임 변화와 한국의 미술관·박물관................................................5\n- 이승미 _ 국립현대미술관 학교연계 교육..................................................................................................15\n- 최윤정 _ CRM관점에서 바라본 국립현대미술관 회원제..........................................................................37\n- 이권호 _ 일본 국공립미술관 제도연구.....................................................................................................49\nⅡ. 현대미술연구\n- 김경운 _ 빌 비올라의 해변 없는 바다.......................................................................................................75\n- 이수연 _ 감각의 인지와 확장 - 백남준의 작업을 중심으로......................................................................85\n- 박수진 _ 재외한인 작가전-일본, 중국, 독립국가연합을 중심으로......................................................... 103\nⅢ. 보존과학 연구\n- 김은진 _ 유화의 배접-역사적 수용과 현대의 대안.........................................

In [62]:
import pickle

with open("docstore_docs.pkl", "wb") as f:
    pickle.dump(docstore_docs, f)

with open("id_map.pkl", "wb") as f:
    pickle.dump(id_map, f)

In [63]:
len(docstore_docs)

924

In [64]:
print("✅ 문서 수:", len(docstore_docs))
print("✅ 벡터 수:", faiss_index.ntotal)

✅ 문서 수: 924
✅ 벡터 수: 924


In [72]:
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke("실내 공기질 측정항목에는 무엇이 있어?")
print(docs[0].page_content[:300])  # 확인용


미술작품의 보존 9
국립현대미술관 실내공기질 측정을 통한 보존환경 연구 
121
120
보이지 않았다. 또한 상관관계 분석 결과 실외공기질과 실내공기질의 상관관계가 매우 적은 것을 
알 수 있었다. 실외 공기질 오염물질 측정 시 검출된 이산화질소(NO₂)와 오존(O₃)은 미술관 
내부에서 검출되지 않았고, 서울관 전시실의 일산화탄소(CO) 및 미세먼지(PM10) 결정계수 값
(r²)의 최대값은 서울관 전시실에서 각각 0.32, 0.35로 비교적 실외공기질과 상관관계가 약한
(Weak) 관계로 나타났다.  
이와 같은 상관관계 분석으


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산 타입 설정 (float16이 일반적)
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용 (메모리 절약)
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) 사용
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_compute_dtype="float16",  # 계산 정밀도 설정
    bnb_4bit_quant_type="nf4",  # NF4 양자화 방식 사용 (효율적)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
)

In [6]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # ✅ 올바른 양자화 설정 적용
    device_map="auto",  # ✅ 자동 GPU 배치
    trust_remote_code=True,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:28<00:00,  7.01s/it]


In [7]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05,

)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
/tmp/ipykernel_910/3455024644.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [8]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 🔹 EXAONE 모델 로드
exaone_model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
exaone_tokenizer = AutoTokenizer.from_pretrained(exaone_model_id)
exaone_model = AutoModelForCausalLM.from_pretrained(
    exaone_model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:21<00:00,  3.12s/it]


In [9]:
from transformers import pipeline

# 파이프라인 생성
exaone_pipe = pipeline(
    "text-generation",
    model=exaone_model,
    tokenizer=exaone_tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)

Device set to use cuda


In [10]:
from langchain.prompts import ChatPromptTemplate

deepseek_template = """
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions **accurately based on the provided document (context)**. 
If the requested information is not found in the document, respond with:
"문서에 해당 정보가 없습니다." 
You must include the **작품 번호** in your response about an artwork.
Please provide a detailed answer.

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, **bold** or *italic* text) to enhance readability.


<|context|>
{context}

<|user|>
Question: {question}

<think>
"""




exaone_template = '''
<|system|>
You are an AI assistant tasked with refining and polishing the provided logical reasoning into a final answer in Korean.  
Your role is to produce a clear, concise, and well-structured response that maintains the original meaning and key details.  
Ensure that your final answer is written in Korean and uses **beautiful Markdown formatting** (e.g., headings, bullet points, **bold** or *italic* text) to enhance readability.  
Focus solely on refining the content without adding any new information.
You must include the **작품 번호** in your response only when it is a question about an artwork.

<|reasoning|>
{reasoning}

<|user|>
Based on the above reasoning, please generate a refined and final answer in Korean.

<|assistant|><think>
'''



# DeepSeek 템플릿 생성
deepseek_prompt = ChatPromptTemplate.from_template(deepseek_template)

# EXAONE 템플릿 생성
exaone_prompt = ChatPromptTemplate.from_template(exaone_template)



In [11]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, bold or italic text or table) to enhance readability.
You must include artwork number.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [ ]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_combined"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [41]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 10,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.3      # 다양성과 관련성 간 균형
    }
)


In [42]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"</think>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [14]:
import re

class MarkdownOutputParser2:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)


/tmp/ipykernel_910/627080515.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/chae/env_0324/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [143]:
from langchain.schema import Document
from langchain_core.retrievers import BaseRetriever
import threading
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import time

# ✅ 멀티쓰레드로 FAISS DB 로드
faiss_db1 = None
faiss_db2 = None

def load_faiss1():
    global faiss_db1
    faiss_db1 = FAISS.load_local("faiss_artworks_0324_json", embedding_model, allow_dangerous_deserialization=True)

def load_faiss2():
    global faiss_db2
    faiss_db2 = FAISS.load_local("faiss_artworks_0324_pdf", embedding_model, allow_dangerous_deserialization=True)

t1 = threading.Thread(target=load_faiss1)
t2 = threading.Thread(target=load_faiss2)

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print("✅ FAISS DB 로드 완료:", round(end - start, 2), "초")


retriever1 = faiss_db1.as_retriever(
    search_kwargs={
        "k": 5,
        "fetch_k": 10,
        "mmr": True,
        "mmr_beta": 0.3
    }
)

retriever2 = faiss_db2.as_retriever(
    search_kwargs={
        "k": 5,
        "fetch_k": 10,
        "mmr": True,
        "mmr_beta": 0.3
    }
)


from langchain_core.retrievers import BaseRetriever
from langchain.schema import Document
from typing import List
from concurrent.futures import ThreadPoolExecutor
from pydantic import Field

class MergedRetriever(BaseRetriever):
    retriever1: BaseRetriever = Field(...)
    retriever2: BaseRetriever = Field(...)
    k: int = Field(default=5)
    tags: List[str] = Field(default_factory=list)
    metadata: dict = Field(default_factory=dict)

    def _get_relevant_documents(self, query: str, **kwargs) -> List[Document]:
        docs = []

        def search1():
            try:
                return self.retriever1._get_relevant_documents(query, run_manager=None)
            except Exception as e:
                print(f"❌ retriever1 오류: {e}")
                return []

        def search2():
            try:
                return self.retriever2._get_relevant_documents(query, run_manager=None)
            except Exception as e:
                print(f"❌ retriever2 오류: {e}")
                return []

        with ThreadPoolExecutor(max_workers=2) as executor:
            futures = [executor.submit(search1), executor.submit(search2)]
            for future in futures:
                docs.extend(future.result())

        # 중복 제거
        seen = set()
        unique_docs = []
        for doc in docs:
            content = doc.page_content.strip()
            if content not in seen:
                seen.add(content)
                unique_docs.append(doc)

        return unique_docs[:self.k]





✅ FAISS DB 로드 완료: 0.07 초


In [144]:
# retriever1, retriever2 생성
retriever1 = faiss_db1.as_retriever(
    search_kwargs={"k": 5, "fetch_k": 10, "mmr": True, "mmr_beta": 0.3}
)
retriever2 = faiss_db2.as_retriever(
    search_kwargs={"k": 5, "fetch_k": 10, "mmr": True, "mmr_beta": 0.3}
)

# ❌ 이렇게 하면 안 됨
# merged_retriever = MergedRetriever(retriever1, retriever2)

# ✅ 이렇게 고쳐야 함
merged_retriever = MergedRetriever(
    retriever1=retriever1,
    retriever2=retriever2,
    k=5
)



In [12]:
from langchain_community.vectorstores import FAISS

faiss_db = FAISS.load_local(
    "faiss_artworks_0324_pdf",  # <- 디렉토리 이름 바꿔도 돼
    embedding_model,
    allow_dangerous_deserialization=True
)

# 3. Retriever 구성
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 3,
        "fetch_k": 6,
        "mmr": False
    }
)

In [15]:
from langchain.schema.runnable import RunnableLambda, RunnableMap

chain = (
    RunnableMap({
        "question": lambda x: x if isinstance(x, str) else x["question"],
        "context": RunnableLambda(lambda x: retriever.get_relevant_documents(x if isinstance(x, str) else x["question"]))
    })
    | prompt
    | exaone_llm
    | MarkdownOutputParser2()
)



In [17]:
query = "미술관의 권고 조도기준은 무엇인가?"
response = chain.invoke({"question": query})
print(response)

/tmp/ipykernel_910/467081321.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "context": RunnableLambda(lambda x: retriever.get_relevant_documents(x if isinstance(x, str) else x["question"]))


### **🔹 모델 응답 결과**

## 미술관 권고 조도기준

국립현대미술관 보존과학 연구지에 따르면, 미술관 및 박물관의 전시물에 대한 권고 조도기준은 다음과 같습니다.

* **빛에 민감한 재료로 이루어진 천연재료 및 고서화, 수채화, 지류 등:** **100 lux 이하**
* **비교적 민감한 재료로 이루어진 유화나 사진 등:** **150 lux 이하**

이러한 기준은 미술 작품이 빛에 의한 변색이나 퇴색을 최소화하도록 돕기 위한 것입니다.


In [155]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: 미술작품의 보존 10
미술품의 보존을 위한 전시실 보존환경 연구
85
84
위치시키고 양극의 밸런싱을 잡아주는 조명을 가운데 설치하여 부드럽게 시선이 이동할 
수 있도록 설치하였다.
조도는 평균 87.5lux으로 측정(Pro'sKitⓇ, MT-4617LED)되어, 천연가죽의 조도기준인 
100lux이하를 만족했으며, UV 측정결과 역시 평균 0㎼/㎡으로 측정되어 (LTLutron, 
UV-340A) 제한 기준인 10㎼/㎡ 이하를 만족한 것으로 나타났다(Figure 8).
Figure 8. Comparison of before and after conservation treatment
삼베 등의 입체 전시작품의 경우, 빛의 음영에 의해 전시물의 형태나 질감 등을 충분히 
보이도록 하는 것이 중요하다(FIgure 9). 스크린 등의 영상 작품의 경우, 영상을 위한 
전시실 내부를 최대한 어둡게 유지하여야 한다.
Figure 9. Comparison of before and after conservation treatment
겨울철 상대습도 관리는 진열장(인클로저)를 제외하고는 사실상 어려운 실정이다.
따라서 작가와 협의 후 미세울음이 발견된 작품을 전시장환경에 맞추는 방향으로 
응급보존처리를 진행하였다. 먼저, 보존처리실 습도를 60%까지 높혀 양가죽의 울음을 
완화시켜주고, 전시장 상대습도인 35~50%에 맞춰 전시할 수 있게 5%씩 상대습도를 
낮추어 가죽의 적응력을 두었다. 최소 상대습도 맞게 적응된 작품에 중성지로 와셔를 
제작하여 재고정하였다(Figure 7).  
그 외 다른 작품에서 실내 온·습도에 의한 간접적인 열화가 발생하지 않은 것으로 
파악되었다.  
Figure 7. Comparison of before and after conservation treatment
3.2.2. 조도 및 자외선
작품의 조명은 눈의 순응을 고려하여 계획하는 것이 중요하다. 특히 전시장 내 미술품을 
전시할 때 전시 공간 전체의 

### DB 2개 연결

In [1]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

/home/chae/env_0324/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [205]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# FAISS 인덱스 로드 (pickle 허용)
faiss_db1 = FAISS.load_local("faiss_artworks_merged_0324", embedding_model,allow_dangerous_deserialization=True)
faiss_db2 = FAISS.load_local("faiss_artworks_0324_pdf2", embedding_model,allow_dangerous_deserialization=True)

# 🔍 index 타입 확인
print("faiss_db1 index type:", type(faiss_db1.index))
print("faiss_db2 index type:", type(faiss_db2.index))


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


faiss_db1 index type: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>
faiss_db2 index type: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>


In [206]:
print("🧩 index 벡터 수:", faiss_db1.index.ntotal)
print("🧩 docstore 문서 수:", len(faiss_db1.docstore._dict))
print("🧩 index_to_docstore_id:", len(faiss_db1.index_to_docstore_id))


🧩 index 벡터 수: 12403
🧩 docstore 문서 수: 12403
🧩 index_to_docstore_id: 12403


In [207]:
print("🧩 index 벡터 수:", faiss_db2.index.ntotal)
print("🧩 docstore 문서 수:", len(faiss_db2.docstore._dict))
print("🧩 index_to_docstore_id:", len(faiss_db2.index_to_docstore_id))


🧩 index 벡터 수: 182
🧩 docstore 문서 수: 182
🧩 index_to_docstore_id: 182


In [208]:
print(type(faiss_db2))

<class 'langchain_community.vectorstores.faiss.FAISS'>


In [209]:
def shift_faiss_ids(faiss_db, offset):
    new_docstore = {}
    for k, v in faiss_db.docstore._dict.items():
        try:
            new_key = str(int(k) + offset)
        except ValueError:
            # 만약 k가 정수로 변환되지 않으면, 새로 고유한 키를 부여하거나 건너뜁니다.
            # 예시: 현재 offset 값을 키로 사용한 후 offset을 1 증가
            new_key = str(offset)
            offset += 1
        new_docstore[new_key] = v
    faiss_db.docstore._dict = new_docstore

    new_index_to_docstore_id = {}
    for i, doc_id in faiss_db.index_to_docstore_id.items():
        try:
            new_doc_id = str(int(doc_id) + offset)
        except ValueError:
            new_doc_id = doc_id
        new_index_to_docstore_id[i] = new_doc_id
    faiss_db.index_to_docstore_id = new_index_to_docstore_id
    return faiss_db


In [210]:
len(faiss_db1.docstore._dict.items())

12403

In [211]:
len(faiss_db2.index_to_docstore_id)

182

In [212]:
# ID 충돌 방지를 위해 faiss_db2의 ID에 offset shift (faiss_db1의 ID 수 만큼 shift)
offset = len(faiss_db1.index_to_docstore_id)
faiss_db2 = shift_faiss_ids(faiss_db2, offset)


In [213]:
len(faiss_db2.index_to_docstore_id)

182

In [214]:
print(type(list(faiss_db2.index_to_docstore_id.keys())[0]))  # 예: <class 'str'> 나 <class 'int'>?

<class 'int'>


In [183]:
faiss_db1.index_to_docstore_id = {i: str(doc_id) for i, doc_id in faiss_db1.index_to_docstore_id.items()}
faiss_db2.index_to_docstore_id = {i: str(doc_id) for i, doc_id in faiss_db2.index_to_docstore_id.items()}

In [215]:
faiss_db1.merge_from(faiss_db2)

In [216]:
faiss_db1.save_local("집에가고싶DB")

In [217]:
# FAISS 인덱스와 docstore의 id 매핑 확인
print("index_to_docstore_id keys:", faiss_db1.index_to_docstore_id.keys())
print("docstore keys:", faiss_db1.docstore._dict.keys())


index_to_docstore_id keys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 21

In [33]:
# 병합!
merged_db.merge_from(llama_db)


merged_db.save_local("faiss_artworks_merged_0324_2")
print("🎉 병합 및 저장 완료 → faiss_artworks_merged_0324_2")

🎉 병합 및 저장 완료 → faiss_artworks_merged_0324_2


In [26]:
final_merged_db = safely_merge_faiss_vectorstores(merged_db, llama_db)
final_merged_db.save_local("faiss_artworks_merged_0324_final")
print("✅ 병합 및 저장 완료!")

AttributeError: 'IndexFlatL2' object has no attribute 'xb'